In [12]:
import json
import pdfplumber
import spacy
import pandas as pd
import numpy as np
import os
import unidecode
import PyPDF2
from tika import parser
from spacy.matcher import PhraseMatcher
from spacy import displacy
from progress_bar.progress_bar import printProgressBar
import torch
import gc
import multiprocessing
import sys
from flair.data import Sentence
from flair.models import SequenceTagger

        

In [13]:
tagger = SequenceTagger.load("flair/ner-english-large")

2022-07-30 00:26:29,202 loading file /home/heider/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-07-30 00:26:40,807 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [14]:
nlp_spa = spacy.load('en_core_web_trf')

In [15]:
#spacy.prefer_gpu()

In [16]:
nlp_compare = spacy.load('en_core_web_lg')
nlp_person = spacy.load('en_core_web_trf')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
#nlp_compare = spacy.load('en_core_web_lg')
#nlp_person = spacy.load('en_core_web_trf')
#nlp_person = spacy.load('es_dep_news_trf')

In [84]:
def clean_break_line(docs):
    new_docs = docs.replace('\n',"")
    return new_docs
    
def clean_spaces_text(text):
    new_text = " ".join(text.split())
    return(new_text)
    
def clean_points_text(docs):
    new_docs = docs.replace('.',"")
    return new_docs

def clean_commas_text(docs):
    new_docs = docs.replace(',',"")
    return new_docs

def clean_accents(text):
    unaccented_string = unidecode.unidecode(text)
    return unaccented_string
    #lower_u_string = unaccented_string.lower()
    #return lower_u_string

def split_authors(text):
    return text.split(";")

def contains_text_list(names, authors):
    df_names = []
    df_authors = []
    df_text = []
    df_lower_authors = []
    for aut in authors:
        for elem in names:
            df_names.append(elem)
            df_lower_authors.append(clean_accents(aut.lower()))
            df_text.append(contains_text(clean_accents(elem.lower()), clean_accents(aut.lower())))
            df_authors.append(aut)
    df = {'names': df_names,'author': df_authors, 'author_lower': df_lower_authors,'contains': df_text}
    df = pd.DataFrame(df)
    #print(df)
    return df

def name_in_title(data_frame, title):
    can_tutor = data_frame.copy()
    can_tutor['is_in_title'] = can_tutor['tutor_lower'].apply(contains_text, args = title)
    
    false_values = can_tutor[can_tutor['is_in_title']==False]
    return false_values
   

def contains_text(text1, text2):
    text_1 = text1.split(" ").copy()
    text_2 = text2.split(" ").copy()
    
    for word_1 in text_1:
        if word_1 != None and len(word_1)>3:
            for word_2 in text_2:
                if word_2!= None and word_1.lower() in word_2.lower():
                    return True
            
    for word_2 in text_2:
        if word_2 != None and len(word_2)>3:
            for word_1 in text_1:
                if word_1!= None and word_2.lower() in word_1.lower():
                    return True
    return False

def pick_candidate_tutor(data_frame, title):
    check_multiple = data_frame.copy()
    
    values = ["Dios", "JOSE GREGORIO HERNANDEZ"]
    values = map(str.lower, values)
    
    #here we delete dios and JOSE GREGORIO HERNANDEZ and return the first name.
    true_values = check_multiple[check_multiple["contains"]==True]
    t_names = true_values["names"]
    t_names.to_list()
    tutor = check_multiple[~check_multiple['names'].isin(t_names.to_list())].copy()
    tutor['tutor_lower'] = tutor['names'].str.lower()
    tutor = tutor[~tutor['tutor_lower'].isin(values)]
    tutor = name_in_title(tutor, title)
    tutor = tutor['names'].to_list()[0]
    return tutor
    

def compare_similarity_t(docs, author, threshold = 0.63):
    aux_docs = docs.copy()
    author = nlp_compare(author)
    for name in aux_docs:
        name_aux = nlp_compare(name)
        if author.similarity(name_aux) >= threshold:
            aux_docs.remove(name)
        #print(name, "<->", author, author.similarity(name_aux))
    return aux_docs

def compare_similarity(docs, author):
    author = nlp_compare(author)
    for name in docs:
        name_aux = nlp_compare(name)
        print(name, "<->", author, author.similarity(name_aux))

def compare_names(docs, names, threshold = 0.63):
    aux = docs.copy()
    if len(names) > 1:
        for name in names:
            aux = compare_similarity_t(aux, name, threshold)
        return aux[0]    
    else:
        aux = compare_similarity_t(aux, names[0], threshold)
        return aux[0]
    
def get_persons(data):
    first_data = data
    doc = nlp_person(first_data)
    get_persons_list = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            get_persons_list.append(ent.text)
    return get_persons_list

def get_persons_flair(data):
    sentence = Sentence(data)
    tagger.predict(sentence)
    
    list_of_names = []
    for label in sentence.get_labels('ner'):
        # print label value and score
        if label.value == "PER":
            #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
            list_of_names.append(label.data_point.text)
    return(list_of_names)

class Tutor():
    def __init__(self, l):
        self.j = 0
        self.l = l
        
    def compare_all_names(self, file_source, author, threshold = 0.63):
        printProgressBar(self.j, self.l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        try:
            parsed_pdf = parser.from_file(file_source)
            data = parsed_pdf['content']
            data = clean_break_line(data)
            data = clean_spaces_text(data)
            data = clean_points_text(data)
            #data = clean_accents_lower(data)
            #data[:8000]
            docs = get_persons(data[:8000])
            docs = list(map(clean_accents, docs)) #clean the accents

            authors = split_authors(author)
            tutor = compare_names(docs, authors, threshold)
            self.j+=1
            return tutor
        except:
            self.j+=1
            return "No tutor"
        
    def compare_all_names_flair(self, file_source, author, title):
        self.j+=1
        printProgressBar(self.j, self.l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        try:
            parsed_pdf = parser.from_file(file_source)
            data = parsed_pdf['content']
            data = clean_break_line(data)
            data = clean_spaces_text(data)
            data = clean_points_text(data)
            docs = data[:8000]
            docs = clean_accents(docs) #clean the accents

            #flair
            sentence = Sentence(docs)
            tagger.predict(sentence)

            list_of_names = []
            for label in sentence.get_labels('ner'):
                # print label value and score
                if label.value == "PER":
                    #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
                    list_of_names.append(label.data_point.text)
            #print(list_of_names)
            check_multiple = contains_text_list(list_of_names, [author])
            can_tutor = pick_candidate_tutor(check_multiple, [title])
            return can_tutor
        except:
            return "No tutor"
                    
            
            
            
        
        
        
    
    

In [85]:
## names
df_tesis = pd.read_csv('data/thesis_7149_with_resumen_school_complex.csv')
df_tesis

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF,isScan,school_simple,school_complex
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
1,249,13-Oct-2011,Análisis y evaluación de la aplicación de tecn...,"González P., Héctor V.",2.61 MB,http://saber.ucv.ve/handle/10872/426,http://saber.ucv.ve/bitstream/10872/426/1/TRAB...,NaN,thesis_pdf_all/249.pdf,True,True,False,escuela de ingenieria de petroleo,Escuela de Ingeniería de Petroleo
2,197,30-Sep-2011,Modelo de participación comunitaria del comité...,"Carrillo C., Nancy; Márquez S., Migdalia",280.68 kB,http://saber.ucv.ve/handle/10872/325,http://saber.ucv.ve/bitstream/10872/325/1/MODE...,NaN,thesis_pdf_all/197.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
3,198,30-Sep-2011,Calidad del cuidado brindado por el profesiona...,"Casaña, Yurima; Florido, Lisbeth; Nieves, Rosa",481.18 kB,http://saber.ucv.ve/handle/10872/326,http://saber.ucv.ve/bitstream/10872/326/1/tesi...,NaN,thesis_pdf_all/198.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
4,199,30-Sep-2011,Programa de educación para la prevención del v...,"Colmenares, Suleimy; Contreras, Rosa Y.; Valer...",483.44 kB,http://saber.ucv.ve/handle/10872/327,http://saber.ucv.ve/bitstream/10872/327/1/TESI...,NaN,thesis_pdf_all/199.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7144,7733,30-Jul-2019,Evaluación y selección de sensores en sistemas...,"Sandia I., Alexandra C.",1.4 MB,http://saber.ucv.ve/handle/10872/20088,http://saber.ucv.ve/bitstream/10872/20088/1/TE...,NaN,thesis_pdf_all/7733.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7145,7734,30-Jul-2019,Diseño de una red industrial MODBUS RTU inalám...,"Temprano G., Alejandro J.",1.35 MB,http://saber.ucv.ve/handle/10872/20089,http://saber.ucv.ve/bitstream/10872/20089/1/TE...,NaN,thesis_pdf_all/7734.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7146,7735,31-Jul-2019,Estimación de la demanda y energía eléctrica e...,"Landaeta R., Ana G.",2.45 MB,http://saber.ucv.ve/handle/10872/20090,http://saber.ucv.ve/bitstream/10872/20090/1/La...,NaN,thesis_pdf_all/7735.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7147,7736,31-Jul-2019,Diseño de un sistema de comunicación entre un ...,"Borges F., Gabriel A.",4.76 MB,http://saber.ucv.ve/handle/10872/20091,http://saber.ucv.ve/bitstream/10872/20091/1/TE...,NaN,thesis_pdf_all/7736.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica


In [96]:
df_tesis_100 = df_tesis.copy()
df_tesis_100

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF,isScan,school_simple,school_complex
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
1,249,13-Oct-2011,Análisis y evaluación de la aplicación de tecn...,"González P., Héctor V.",2.61 MB,http://saber.ucv.ve/handle/10872/426,http://saber.ucv.ve/bitstream/10872/426/1/TRAB...,NaN,thesis_pdf_all/249.pdf,True,True,False,escuela de ingenieria de petroleo,Escuela de Ingeniería de Petroleo
2,197,30-Sep-2011,Modelo de participación comunitaria del comité...,"Carrillo C., Nancy; Márquez S., Migdalia",280.68 kB,http://saber.ucv.ve/handle/10872/325,http://saber.ucv.ve/bitstream/10872/325/1/MODE...,NaN,thesis_pdf_all/197.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
3,198,30-Sep-2011,Calidad del cuidado brindado por el profesiona...,"Casaña, Yurima; Florido, Lisbeth; Nieves, Rosa",481.18 kB,http://saber.ucv.ve/handle/10872/326,http://saber.ucv.ve/bitstream/10872/326/1/tesi...,NaN,thesis_pdf_all/198.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
4,199,30-Sep-2011,Programa de educación para la prevención del v...,"Colmenares, Suleimy; Contreras, Rosa Y.; Valer...",483.44 kB,http://saber.ucv.ve/handle/10872/327,http://saber.ucv.ve/bitstream/10872/327/1/TESI...,NaN,thesis_pdf_all/199.pdf,True,True,False,escuela de enfermeria,Escuela de Enfermería
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7144,7733,30-Jul-2019,Evaluación y selección de sensores en sistemas...,"Sandia I., Alexandra C.",1.4 MB,http://saber.ucv.ve/handle/10872/20088,http://saber.ucv.ve/bitstream/10872/20088/1/TE...,NaN,thesis_pdf_all/7733.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7145,7734,30-Jul-2019,Diseño de una red industrial MODBUS RTU inalám...,"Temprano G., Alejandro J.",1.35 MB,http://saber.ucv.ve/handle/10872/20089,http://saber.ucv.ve/bitstream/10872/20089/1/TE...,NaN,thesis_pdf_all/7734.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7146,7735,31-Jul-2019,Estimación de la demanda y energía eléctrica e...,"Landaeta R., Ana G.",2.45 MB,http://saber.ucv.ve/handle/10872/20090,http://saber.ucv.ve/bitstream/10872/20090/1/La...,NaN,thesis_pdf_all/7735.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica
7147,7736,31-Jul-2019,Diseño de un sistema de comunicación entre un ...,"Borges F., Gabriel A.",4.76 MB,http://saber.ucv.ve/handle/10872/20091,http://saber.ucv.ve/bitstream/10872/20091/1/TE...,NaN,thesis_pdf_all/7736.pdf,True,True,False,escuela de ingenieria electrica,Escuela de Ingeniería Eléctrica


In [97]:
l = len(df_tesis_100)
l

7149

In [ ]:
tutores = Tutor(l)
l = len(df_tesis_100)
compare_fn = np.vectorize(tutores.compare_all_names_flair, otypes=[np.ndarray])
values_c = compare_fn('../'+ df_tesis_100['path'], df_tesis_100['thesis_author'], df_tesis_100['thesis_title'])
df_tesis_100['thesis_tutor'] = values_c.tolist()

In [ ]:
df_tesis_100.to_csv("./data/thesis_tutor_7149.csv", index=False)

In [80]:
df_tesis_100['thesis_tutor']

0    Julio Molina
Name: thesis_tutor, dtype: object

In [11]:
df_tesis_100.to_csv("./data/thesis_tutor_5.csv", index=False)

In [51]:
thesis_number = '1'
file_source = '../'+'thesis_pdf_all/' + thesis_number + '.pdf'

parsed_pdf = parser.from_file(file_source)
data = parsed_pdf['content']
data = clean_break_line(data)
data = clean_spaces_text(data)
data = clean_points_text(data)
data = clean_commas_text(data)
#data = clean_accents_lower(data)
data[:8000]

"TRABAJO ESPECIAL DE GRADO ESTUDIO DE SOBRETENSIONES POR MANIOBRA DEBIDO A ENERGIZACIÓN DE BANCOS DE CONDENSADORES Y TRANSFORMADORES EN LÍNEAS DE TRANSMISIÓN DE 115kV Tutor Académico: Ing Julio Molina Presentado ante la Ilustre Universidad Central de Venezuela por el Br De Gregorio S Luis T para optar al titulo de Ingeniero Electricista Caracas 2009 ii CONSTANCIA DE APROBACIÓN Caracas 13 de mayo de 2009 Los abajo firmantes miembros del Jurado designado por el Consejo de Escuela de Ingeniería Eléctrica para evaluar el Trabajo Especial de Grado presentado por el Bachiller Luis T De Gregorio S titulado: “ESTUDIO DE SOBRETENSIONES POR MANIOBRA DEBIDO A ENERGIZACIÓN DE BANCOS DE CONDENSADORES Y TRANSFORMADORES EN LÍNEAS DE TRANSMISIÓN DE 115kV” Consideran que el mismo cumple con los requisitos exigidos por el plan de estudios conducente al Título de Ingeniero Electricista en la mención de Potencia y sin que ello signifique que se hacen solidarios con las ideas expuestas por el autor lo decl

In [52]:
author = df_tesis[df_tesis['index'] == int(thesis_number)]
author = author['thesis_author']
author = author.to_list()
author = split_authors(author[0])
author

['De Gregorio S., Luis T.']

In [53]:
title = df_tesis[df_tesis['index'] == int(thesis_number)]
title = title['thesis_title'].to_list()
title = title
title

['Estudio de sobretensiones por maniobra debido a energización de bancos de condensadores y transformadores en líneas de transmisión de 115kV']

In [59]:
sentence = Sentence(data[:8000])

In [60]:
tagger.predict(sentence)

In [56]:
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    #print(entity.value)
    print(entity)

The following NER tags are found:
Span[28:30]: "Julio Molina" → PER (0.9999)
Span[33:38]: "Ilustre Universidad Central de Venezuela" → ORG (1.0)
Span[41:46]: "De Gregorio S Luis T" → PER (0.9979)
Span[53:54]: "Caracas" → LOC (1.0)
Span[59:60]: "Caracas" → LOC (1.0)
Span[74:80]: "Consejo de Escuela de Ingeniería Eléctrica" → ORG (1.0)
Span[91:96]: "Luis T De Gregorio S" → PER (0.9998)
Span[173:175]: "Dios Padre" → PER (0.8139)
Span[222:223]: "Dios" → PER (0.9987)
Span[249:250]: "Dios" → PER (0.9959)
Span[280:284]: "María de los Ángeles" → PER (0.9999)
Span[303:304]: "Dios" → PER (0.9997)
Span[352:353]: "Henry" → PER (0.9256)
Span[356:358]: "María Fernanda" → PER (1.0)
Span[364:365]: "Dios" → PER (0.9996)
Span[400:402]: "Ángel González" → PER (1.0)
Span[402:405]: "José Benancio Benavides" → PER (1.0)
Span[405:408]: "Juan Miguel Otero" → PER (1.0)
Span[408:410]: "Víctor Bermúdez" → PER (1.0)
Span[411:413]: "Yanis Sánchez" → PER (1.0)
Span[437:440]: "Ing Julio Molina" → PER (0.9999)
Span[4

In [62]:
list_of_names = []
for label in sentence.get_labels('ner'):
    # print label value and score
    if label.value == "PER":
        #print(f'the text of label.data_point is: "{label.data_point.text}"\n')
        list_of_names.append(label.data_point.text)
print(list_of_names)
    #print(f'label.value is: "{label.value}"')
    #print(f'label.score is: "{label.score}"')
    # access the data point to which label attaches and print its text
    #print(f'the text of label.data_point is: "{label.data_point.text}"\n')

['Julio Molina', 'De Gregorio S Luis T', 'Luis T De Gregorio S', 'Dios Padre', 'Dios', 'Dios', 'María de los Ángeles', 'Dios', 'Henry', 'María Fernanda', 'Dios', 'Ángel González', 'José Benancio Benavides', 'Juan Miguel Otero', 'Víctor Bermúdez', 'Yanis Sánchez', 'Ing Julio Molina', 'v', 'De Gregorio S Luis', 'Julio Molina']


In [36]:
compare_similarity(list_of_names, author[0])

Julio Molina <-> De Gregorio S., Luis T. 0.5524505221316749
De Gregorio S Luis T <-> De Gregorio S., Luis T. 0.8766791133385216
Luis T De Gregorio S <-> De Gregorio S., Luis T. 0.8766791183001341
Dios Padre <-> De Gregorio S., Luis T. 0.43350252091602565
Dios <-> De Gregorio S., Luis T. 0.36197559504542337
Dios <-> De Gregorio S., Luis T. 0.36197559504542337
María de los Ángeles <-> De Gregorio S., Luis T. 0.5852971466988179
Dios <-> De Gregorio S., Luis T. 0.36197559504542337
Henry <-> De Gregorio S., Luis T. 0.4579960933893402
María Fernanda <-> De Gregorio S., Luis T. 0.3351272088552749
Dios <-> De Gregorio S., Luis T. 0.36197559504542337
Ángel González <-> De Gregorio S., Luis T. 0.24618959322419287
José Benancio Benavides <-> De Gregorio S., Luis T. 0.45269990713767955
Juan Miguel Otero <-> De Gregorio S., Luis T. 0.5869483456306895
Víctor Bermúdez <-> De Gregorio S., Luis T. 0.0
Yanis Sánchez <-> De Gregorio S., Luis T. 0.0
Ing Julio Molina <-> De Gregorio S., Luis T. 0.599930894

/tmp/ipykernel_24894/2974367896.py:98: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [37]:
compare_names(list_of_names, author, threshold = 0.43)

/tmp/ipykernel_24894/2974367896.py:89: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'De Gregorio S Luis T'

In [38]:
check_multiple = contains_text_list(list_of_names, author)
can_tutor = pick_candidate_tutor(check_multiple, title)
can_tutor


/tmp/ipykernel_24894/2974367896.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tutor['tutor_lower'] = tutor['names'].str.lower().copy()


'Julio Molina'

In [39]:
can_tutor['names'].to_list()[0]

TypeError: string indices must be integers

In [671]:
compare_names(can_tutor['names'].to_list(), author, threshold = 0.5)

'Aura Perez'

In [663]:
can_tutor['is_in_title'] = can_tutor['tutor_lower'].apply(contains_text, args = title)
can_tutor

,names,author,author_lower,contains,tutor_lower,is_in_title
6,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
7,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
10,Mylanyela Rosana,"Carrillo C., Nancy","carrillo c., nancy",False,mylanyela rosana,False
11,Omar,"Carrillo C., Nancy","carrillo c., nancy",False,omar,False
13,Venus,"Carrillo C., Nancy","carrillo c., nancy",False,venus,False
16,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
17,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
22,Aura Perez,"Carrillo C., Nancy","carrillo c., nancy",False,aura perez,False
23,Jaime Garrido,"Carrillo C., Nancy","carrillo c., nancy",False,jaime garrido,False
30,Aura Perez,"Márquez S., Migdalia","marquez s., migdalia",False,aura perez,False


In [566]:
contains_text("INGRID PEREZ", title)

False

In [503]:
compare_names(can_tutor['names'].to_list(), author)

'INGRID PEREZ'

In [345]:
author[1].split(" ")

['', 'Contreras,', 'Rosa', 'Y.']

In [346]:
list_of_names[0].split(" ")

['JOSE', 'GREGORIO', 'HERNANDEZ']

In [347]:

contains_text(list_of_names[0], author[0])

False

In [348]:
pd.set_option('display.max_rows', None)
check_multiple = contains_text_list(list_of_names, author)

In [349]:
check_multiple_names = check_multiple['names']
#check_multiple_names.to_list()

In [350]:
true_values = check_multiple[check_multiple["contains"]==True]
t_names = true_values["names"]
t_names.to_list()

['Colmenares Suleimy',
 'Colmenares Suleimy',
 'Colmenares Suleimy',
 'Colmenares Suleimy Contreras',
 'Colmenares Suleimy Contreras',
 'Contreras Rosa Yadira',
 'Contreras Rosa Yadira',
 'Rosa Yadira',
 'Colmenares Suleimy Contreras',
 'Rosa Yadira',
 'Colmenares Suleimy Contreras',
 'Rosa Yadira Valera',
 'Valera Carmen Elena',
 'Valera Carmen Elena',
 'Carmen Valera',
 'Valera Carmen Elena',
 'Rosa Yadira Valera',
 'Carmen Elena']

In [361]:
tutor = check_multiple[~check_multiple['names'].isin(t_names.to_list())]
tutor = tutor[lower(tutor["names"]) != "JOSE GREGORIO HERNANDEZ"]
tutor[tutor["names"] != "Dios"]

NameError: name 'lower' is not defined

In [48]:
first_data = data[:8000]
doc = nlp_person(first_data)
get_persons = []
for ent in doc.ents:
    if ent.label_ == 'PERSON':
        get_persons.append(ent.text)
    
get_persons

['Rosa Yadira Valera',
 'Colmenares Suleimy',
 'Contreras Rosa Yadira',
 'Valera Carmen Elena',
 'Rosa Yadira',
 'Dios',
 'Yadira',
 'Valera Carmen Elena Para',
 'Rosa Yadira Valera',
 'Carmen Elena']

In [49]:
get_persons = list(map(clean_accents_lower, get_persons))

In [57]:
## names
df_tesis = pd.read_csv('data/thesis_7149_with_resumen_school_complex.csv')
authors = df_tesis[df_tesis['index'] == int(thesis_number)]['thesis_author']
authors
authors

4    Colmenares, Suleimy; Contreras, Rosa Y.; Valer...
Name: thesis_author, dtype: object

In [59]:
split_fn = np.vectorize(split_authors)
split_fn(authors).tolist()

ValueError: setting an array element with a sequence

In [465]:
split_authors(df_tesis['thesis_author'][1])

['González P., Héctor V.']

In [466]:
df_tesis['thesis_author']

0                                 De Gregorio S., Luis T.
1                                  González P., Héctor V.
2                Carrillo C., Nancy; Márquez S., Migdalia
3          Casaña, Yurima; Florido, Lisbeth; Nieves, Rosa
4       Colmenares, Suleimy; Contreras, Rosa Y.; Valer...
                              ...                        
7144                              Sandia I., Alexandra C.
7145                            Temprano G., Alejandro J.
7146                                  Landaeta R., Ana G.
7147                                Borges F., Gabriel A.
7148                          Utrera Ávila, Joselin Marly
Name: thesis_author, Length: 7149, dtype: object

In [467]:
tutores = Tutor

In [469]:
compare_fn = np.vectorize(tutores.compare_all_names)
compare_fn()

In [435]:
top10_clean[int(thesis_number)-1]

'Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L'

In [436]:
names = list(map(clean_accents_lower, split_authors(top10_clean[int(thesis_number)-1])))
names 

['Adrian Mogollon Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']

In [437]:
names = split_authors(top10_clean[int(thesis_number)-1])
names

['Adrian Mogollón Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']

In [438]:
compare_similarity(get_persons, names[0])

Yusmary TUTOR <-> Adrian Mogollón Julia 0.11692980703110051
Angel Moros R. <-> Adrian Mogollón Julia 0.45962000177794815
Echezuria <-> Adrian Mogollón Julia 0.0
Yusmary <-> Adrian Mogollón Julia 0.0
Moros <-> Adrian Mogollón Julia 0.15718233851904745
Angel ivDEDICATORIA <-> Adrian Mogollón Julia 0.37152739068934554
Mil Gracias <-> Adrian Mogollón Julia 0.26608767915043785
Julita <-> Adrian Mogollón Julia 0.0
Yusmy <-> Adrian Mogollón Julia 0.0
Yusmary vAGRADECIMIENTOS <-> Adrian Mogollón Julia 0.0
Carmen Barrios <-> Adrian Mogollón Julia 0.5507273551865355
Profesor Angel Moros <-> Adrian Mogollón Julia 0.41511536453209774
Aida Damas <-> Adrian Mogollón Julia 0.34371654321983364
Angel Moros R. RESUMEN <-> Adrian Mogollón Julia 0.459620014454642


/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [439]:
compare_names(get_persons, names, threshold=.46)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [440]:
fn_compare_names = np.vectorize(compare_names)

In [ ]:
values_compare = fn_compare_names

In [431]:
compare_names(get_persons, names)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [407]:
compare_names(get_persons, names)

/tmp/ipykernel_5742/397893262.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


'Yusmary TUTOR'

In [116]:
compare_similarity_t(get_persons, names[1])

/tmp/ipykernel_5742/1498625836.py:27: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if author.similarity(name_aux) >= threshold:


['Angel Moros R',
 'Yusmary Valero G',
 'Yusmy',
 'Alejandra Gonzalez',
 'Angel Moros',
 'Yusmary Valero',
 'Angel Moros R RESUMEN']

In [117]:
compare_similarity(get_persons, names[2])

Angel Moros R <->  Valero Goncalves Yusmary L 0.32039604916347836
Yusmary Valero G <->  Valero Goncalves Yusmary L 0.6697135616296339
Yusmy <->  Valero Goncalves Yusmary L 0.0
Alejandra Gonzalez <->  Valero Goncalves Yusmary L 0.47268456182808843
Angel Moros <->  Valero Goncalves Yusmary L 0.25236357407492055
Yusmary Valero <->  Valero Goncalves Yusmary L 0.7156886585504478
Angel Moros R RESUMEN <->  Valero Goncalves Yusmary L 0.3203960671539872


/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [118]:
compare_similarity_t(get_persons, names[2])

['Angel Moros R',
 'Yusmy',
 'Alejandra Gonzalez',
 'Angel Moros',
 'Angel Moros R RESUMEN']

In [40]:
#file_source = '../'+'thesis_pdf_all/1.pdf'

#parsed_pdf = parser.from_file(file_source)
#data = parsed_pdf['content']

#data

In [42]:
#data = data.replace('\n',"")
#data = clean_spaces_text(data)
#data

In [43]:
#first_data = data[:600]
#first_data

In [8]:
doc = nlp_spa("Margarita camina sobre la torta")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Margarita camina 0 16 PERSON


In [9]:
doc = nlp_spa(first_data)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Luis T. 288 295 PERSON
2009 352 356 DATE
CONSTANCIA DE APROBACIÓN 360 384 PERSON
13 de mayo de 2009 394 412 DATE
Luis T. De Greg 585 600 PERSON


In [23]:
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_spa = spacy.load('es_core_news_sm')
#nlp_spa = spacy.load('en_core_web_sm')
#nlp_person = spacy.load('es_core_news_lg')
nlp_compare = spacy.load('en_core_web_lg')
nlp_person = spacy.load('en_core_web_trf')
#nlp_spa = spacy.load('es_dep_news_trf')

In [24]:
thesis_number = '3'
file_source = '../'+'thesis_pdf_all/' + thesis_number + '.pdf'

parsed_pdf = parser.from_file(file_source)
data = parsed_pdf['content']
data = clean_break_line(data)
data = clean_spaces_text(data)
data = clean_points_text(data)
data[:8000]

'UNIVERSIDAD CENTRAL DE VENEZUELAUNIVERSIDAD CENTRAL DE VENEZUELA FACULTAD DE HUMANIDADES Y EDUCACIÓN ESCUELA DE BIBLIOTECOLOGÍA Y ARCHIVOLOGÍA LA PRODUCCIÓN DE CONOCIMIENTO: UN ANÁLISIS MÉTRICO EN EL PROGRAMA DE INVESTIGACIÓN Y DESARROLLO DEL IDEC-FAU-UCV Trabajo de licenciatura presentado como requisito de la Escuela de Bibliotecología y Archivología de la Universidad Central de Venezuela para optar al Título de Licenciadas en Bibliotecología y Licenciado en Archivología PRESENTADO: Adrián M Julia Echezuría P Jorge Valero G Yusmary TUTOR: Prof Dr Ángel Moros R Caracas octubre 2008 ADRIÁN Julia Variabilidad de productores y producción de conocimiento del Programa de Investigación y Desarrollo del IDEC-FAU-UCV / Julia Adrián M Jorge Echezuría P Yusmary Valero G ; Tutor Prof Dr Ángel Moros R— Caracas 2008 x 107 h; il ; 30cm Trabajo de Licenciatura (Bibliotecología y Archivología) – Universidad Central de Venezuela 2008 1 ANALISIS MÉTRICO 2 BIBLIOMETRÍA 3 PRODUCCIÓN DE CONOCIMIENTO 4 PID

In [25]:
first_data = data[:8000]
doc = nlp_person(first_data)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Adrián M 489 497 PERSON
Julia Echezuría 498 513 PERSON
Jorge Valero G Yusmary TUTOR 516 544 PERSON
Ángel Moros R 554 567 PERSON
Yusmary Valero G 754 770 PERSON
Universidad Central de Venezuela 893 925 ORG
Yusmy 1544 1549 PERSON
Jorge 1764 1769 PERSON
Alejandra González 2140 2158 PERSON
Carmen Barrios 2198 2212 PERSON
Ángel Moros 2266 2277 PERSON
José López 2410 2420 PERSON
Isabel Bello 2664 2676 PERSON
Instituto de Desarrollo Experimental de la Construcción ( 3943 4000 ORG
Programa de Investigación y Desarrollo ( 4013 4053 ORG
1985-2007 5702 5711 DATE
2000-2007 5777 5786 DATE
1985-2007 5864 5873 DATE
2000-2007 5952 5961 DATE
1985-2007 6018 6027 DATE
Julia Adrián Jorge Echezuria 6465 6493 PERSON
Yusmary Valero 6494 6508 PERSON
Ángel Moros R RESUMEN 6516 6537 PERSON
2000-2007 6734 6743 DATE
1985-2006 6817 6826 DATE
2 6967 6968 CARDINAL
3 6999 7000 CARDINAL
two 7513 7516 CARDINAL
2000-2007 7591 7600 DATE
1995-2006 7677 7686 DATE
three 7709 7714 CARDINAL
1 7723 7724 CARDINAL
2 7816 7817 CA

In [27]:
get_persons = []
for ent in doc.ents:
    if ent.label_ == 'PERSON':
        get_persons.append(ent.text)
    
get_persons

['Adrián M',
 'Julia Echezuría',
 'Jorge Valero G Yusmary TUTOR',
 'Ángel Moros R',
 'Yusmary Valero G',
 'Yusmy',
 'Jorge',
 'Alejandra González',
 'Carmen Barrios',
 'Ángel Moros',
 'José López',
 'Isabel Bello',
 'Julia Adrián Jorge Echezuria',
 'Yusmary Valero',
 'Ángel Moros R RESUMEN']

In [28]:
## names
df_tesis = pd.read_csv('data/url_thesis_8211_with_pdf_scan.csv')
df_tesis

,index,thesis_year,thesis_title,thesis_author,size,thesis_link,pdf_link,resumen,path,exist,isPDF
0,1,2009,Estudio de sobretensiones por maniobra debido ...,"De Gregorio S., Luis T.",6.77 MB,http://saber.ucv.ve/handle/10872/17712,http://saber.ucv.ve/bitstream/10872/17712/1/TE...,NaN,thesis_pdf_all/1.pdf,True,True
1,2,2010,La Economía del Ocio como una alternativa de e...,"Rivera, Elizabeth",2 MB,http://saber.ucv.ve/handle/10872/16245,http://saber.ucv.ve/bitstream/10872/16245/1/TG...,NaN,thesis_pdf_all/2.pdf,True,True
2,3,27-May-2011,La producción de conocimiento : un análisis mé...,"Adrian Mogollón, Julia; Echezuria Perozo, Jorg...",995.92 kB,http://saber.ucv.ve/handle/10872/29,http://saber.ucv.ve/bitstream/10872/29/1/TL-EB...,NaN,thesis_pdf_all/3.pdf,True,True
3,4,27-May-2011,Desarrollo de un algoritmo computacional para ...,"D'Amico, Walter F.",506.32 kB,http://saber.ucv.ve/handle/10872/32,http://saber.ucv.ve/bitstream/10872/32/1/Tesis...,NaN,thesis_pdf_all/4.pdf,True,True
4,5,27-May-2011,Evaluación geofísica integral de los conos alu...,"Cragno, Analia V.; Romero, Marcos A.",3.02 MB,http://saber.ucv.ve/handle/10872/31,http://saber.ucv.ve/bitstream/10872/31/1/Evalu...,NaN,thesis_pdf_all/5.pdf,True,True
...,...,...,...,...,...,...,...,...,...,...,...
8168,8207,4-Dec-2020,Elaboración de un manual de procedimientos e i...,"Armas R., Rodolfo J.",924.37 kB,http://saber.ucv.ve/handle/10872/20906,http://saber.ucv.ve/bitstream/10872/20906/1/TE...,NaN,thesis_pdf_all/8207.pdf,True,True
8169,8208,8-Mar-2021,Condiciones de vida de los estudiantes de Geoq...,"Pérez, Andrea; Pinto, Hendrith",7.21 MB,http://saber.ucv.ve/handle/10872/20969,http://saber.ucv.ve/bitstream/10872/20969/1/CO...,NaN,thesis_pdf_all/8208.pdf,True,True
8170,8209,8-Mar-2021,Análisis de los elementos pospornográficos de ...,"Belandria, Sofía",1.13 MB,http://saber.ucv.ve/handle/10872/20970,http://saber.ucv.ve/bitstream/10872/20970/1/So...,NaN,thesis_pdf_all/8209.pdf,True,True
8171,8210,11-Mar-2021,La Fecundidad Adolescente en Venezuela vista a...,"Simao, Edith Carolina",3.56 MB,http://saber.ucv.ve/handle/10872/20973,http://saber.ucv.ve/bitstream/10872/20973/1/TF...,NaN,thesis_pdf_all/8210.pdf,True,True


In [29]:
top10_authors = df_tesis['thesis_author'][:10]
top10_authors = top10_authors.values.tolist()
top10_authors

['De Gregorio S., Luis T.',
 'Rivera, Elizabeth',
 'Adrian Mogollón, Julia; Echezuria Perozo, Jorge E.; Valero Goncalves, Yusmary L.',
 "D'Amico, Walter F.",
 'Cragno, Analia V.; Romero, Marcos A.',
 'Fernández, Fernando J.',
 'Echezuria Perozo, Jorge E.; Rondon, María C.; Gutierrez Luna, Melany',
 'Silva P., Daniel J.',
 'Silva, Carolina del V.',
 'Suárez B., Efren J.']

In [30]:
top10_authors[2]

'Adrian Mogollón, Julia; Echezuria Perozo, Jorge E.; Valero Goncalves, Yusmary L.'

In [31]:
vec_points = np.vectorize(clean_points_text) 

In [32]:
top10_clean = vec_points(top10_authors)
top10_clean = top10_clean.tolist()

In [107]:
def compare_similarity(docs, author):
    author = nlp_compare(author)
    for name in docs:
        name_aux = nlp_compare(name)
        print(name, "<->", author, author.similarity(name_aux))

In [34]:
compare_similarity(get_persons, str(top10_clean[2]))

Adrián M <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.33937008053670276
Julia Echezuría <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.5670012875522249
Jorge Valero G Yusmary TUTOR <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.7070172418471509
Ángel Moros R <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.40224459980277893
Yusmary Valero G <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.5975819452444061
Yusmy <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.0
Jorge <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.676725298944286
Alejandra González <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.35765417137227434
Carmen Barrios <-> Adrian Mogollón Julia; Echezuria Perozo Jorge E; Valero Goncalves Yusmary L 0.47139572

/tmp/ipykernel_5742/1030706800.py:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(name, "<->", author, author.similarity(name_aux))


In [35]:
top10_clean[2].split(";")

['Adrian Mogollón Julia',
 ' Echezuria Perozo Jorge E',
 ' Valero Goncalves Yusmary L']